In [1]:
from bigml.api import BigML

In [2]:
from pandas import read_csv

In [3]:
df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-train.csv', index_col=0)
df = df.fillna(0)
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0' 
df.to_csv('train_full.csv')

In [4]:
df = read_csv('https://oml-data.s3.amazonaws.com/kaggle-give-me-credit-test.csv', index_col=0)
df = df.fillna(0)
df['MonthlyPerPerson'] = df['MonthlyIncome'] / ( df['NumberOfDependents'] + 1 )
df.loc[df.age > 80, 'isOld'] = '1' 
df.loc[df.age <= 80, 'isOld'] = '0' 
df.to_csv('test_full.csv')

In [5]:
api = BigML('MAXIMEFAVOT', '69757fb108f0bd7a8678e0b66e3de66ae6478c81', project='project/5d9e081659f5c320e1000597')

In [6]:
source_train = api.create_source('train_full.csv')
trainfull_dataset = api.create_dataset(source_train)

In [7]:
source_test = api.create_source('test_full.csv')
testfull_dataset = api.create_dataset(source_test)

In [8]:
train_dataset = api.create_dataset(
    source_train, {"name": "Training_split",
                     "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    source_train, {"name": "Test_split",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [9]:
ensemble = api.create_ensemble(train_dataset, {"name": "ensemble", "objective_field": "SeriousDlqin2yrs"})

In [10]:
evaluation = api.create_evaluation(ensemble, test_dataset)

In [11]:
batch_prediction = api.create_batch_prediction(ensemble, testfull_dataset,{"all_fields": True, "probabilities": True})

In [22]:
api.download_batch_prediction(batch_prediction, filename='BatchPredictionEnsemble.csv')

'BatchPredictionEnsemble.csv'

In [23]:
df = read_csv('BatchPredictionEnsemble.csv', index_col=0)

In [24]:
df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,MonthlyPerPerson,isOld,SeriousDlqin2yrs.1,0 probability,1 probability
field1,,,,,,,,,,,,,,,,
1,0,0.885519,43,0,0.177513,5700,4,0,0,0,0,5700.000000,0,0,0.93568,0.06432
2,0,0.463295,57,0,0.527237,9141,15,0,4,0,2,3047.000000,0,0,0.95497,0.04503
3,0,0.043275,59,0,0.687648,5083,12,0,1,0,2,1694.333333,0,0,0.99042,0.00959
4,0,0.280308,38,1,0.925961,3200,7,0,2,0,0,3200.000000,0,0,0.92624,0.07376
5,0,1.000000,27,0,0.019917,3865,4,0,0,0,1,1932.500000,0,0,0.86970,0.13030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101499,0,0.282653,24,0,0.068522,1400,5,0,0,0,0,1400.000000,0,0,0.96733,0.03267
101500,0,0.922156,36,3,0.934217,7615,8,0,2,0,4,1523.000000,0,0,0.69909,0.30091
101501,0,0.081596,70,0,836.000000,0,3,0,0,0,0,0.000000,0,0,0.99593,0.00407


In [25]:
df2 = df[['1 probability']]

In [26]:
df2

,1 probability
field1,
1,0.06432
2,0.04503
3,0.00959
4,0.07376
5,0.13030
...,...
101499,0.03267
101500,0.30091
101501,0.00407


In [27]:
df2.index.names = ['Id'] 
df2.rename(columns={"1 probability": "Probability"}, inplace= True)

In [28]:
df2

,Probability
Id,
1,0.06432
2,0.04503
3,0.00959
4,0.07376
5,0.13030
...,...
101499,0.03267
101500,0.30091
101501,0.00407


In [30]:
df2.to_csv('Kaggle-submission.csv')

In [31]:
import kaggle
submission_file = "Kaggle-submission.csv"
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMesomeCredit")

100%|██████████| 1.66M/1.66M [00:05<00:00, 337kB/s] 


Successfully submitted to Give Me Some Credit